# Rule Based System applied to Tarot

### Imports

In [39]:
import random
import pandas as pd
import statistics

## Game Modelisation

Are needed :
- trump cards + the fool + oudlers
- figures = King, Queen, Paladin, Jack
- whiteskins = 1 to 10
- the dog (neither first 3 nor last but not important here)
- colors = hearts, clubs, diamonds and spades
- Hand
- taker


Steps :
- Dealing
- Biding
- Handful
- Play of the card

## Cards & Bids Definition

In [40]:
### CARDS ###
FIGURES = ['J','P','Q','K']
FIGURES_NAMES = ["Jack","Knight","Queen","King"]
NON_FIGURES = ['1','2','3','4','5','6','7','8','9','10']
CARD_VALUES = NON_FIGURES + FIGURES

COLOR_VALUES = [ 'H', 'C', 'D', 'S']
COLOR_NAMES = ["Hearts","Clubs","Diamonds","Spades"]

TRUMPS = [ i for i in range(0,22)]
OUDLERS = [0, 1, 21]

COLOR_CARDS = [ COLOR_VALUES[j]+str(CARD_VALUES[i]) for j in range(4) for i in range(len(CARD_VALUES))]

PLAYING_CARDS = TRUMPS + COLOR_CARDS

### BIDS ###

BIDS=['PASS','TAKE','PUSH','GUARD','GUARD WITHOUT','GUARD AGAINST']    
BID_VALUES=[0,10,20,40,60,80]
# quantiles are computed from 1345 takings out of 10K bidings
QUANTILES=[1.005, 1.015, 1.035, 1.045, 1.065, 1.075, 1.105, 1.135, 1.185]
ROLES=['TAKER','OPENER','NEUTRAL','RELAUNCHER']

## Cards Recognition

In [41]:
### CARD FUNCTIONS ###
def get_card_role(card):
    # TODO : check else clause validity
    return card[1:] if card not in TRUMPS else card

def is_figure(card):
    return get_card_role(card) in FIGURES

def is_non_figure(card):
    return get_card_role(card) in NON_FIGURES

def is_trump(card):
    return type(card) is int

def is_oudler(card):
    return card in OUDLERS

def get_color(card):
    return card[0] if not is_trump(card) else None


### TESTS ###
sampling = random.sample(PLAYING_CARDS,2)

for card in sampling:
    print(f"\n=== Card : {card} ===\nCard value : {get_card_role(card)}\nis figure : {is_figure(card)}\nis not figure {is_non_figure(card)}\nis trump : {is_trump(card)}\nis oudler : {is_oudler(card)}\nhas color : {get_color(card)}")


=== Card : 8 ===
Card value : 8
is figure : False
is not figure False
is trump : True
is oudler : False
has color : None

=== Card : HP ===
Card value : P
is figure : True
is not figure False
is trump : False
is oudler : False
has color : H


## Hand Management : Sorting, Showing, Information

In [42]:
# UTILITY FUNCTION
def get_index(card):
    '''Utility function used to sort cards.'''
    return PLAYING_CARDS.index(card)

# UTILITY FUNCTION
def get_card_value(card):
    return (get_index(card)-22)%14+1 if not is_trump(card) else 14+card

# UTILITY FUNCTION
def sort_cards(hand):
    return hand.sort(key=get_index)

# UI FUNCTION
def to_string(card):
    if is_trump(card):
        if card != 0:
            return(f"Trump {card}")
        else:
            return(f"Trump Fool")
    else:
        return(f"{COLOR_NAMES[COLOR_VALUES.index(card[0])]} {FIGURES_NAMES[FIGURES.index(card[1:])] if is_figure(card) else card[1:]}")

# UI FUNCTION
def show_cards(cards):
    for card in cards:
        print(to_string(card))

# HAND FUNCTION
def get_trump_cards(hand):
    return [card for card in hand if is_trump(card)]
def get_trump_count(hand):
    return sum([1 for card in get_trump_cards(hand)])

# HAND FUNCTION
def get_color_cards(hand,color):
    return [card for card in hand if get_color(card)==color]
def get_color_count(hand,color):
    return sum([1 for card in get_color_cards(hand,color)])

### TESTS ###
# TODO implement tests

## Trick Management

In [43]:
def holder(trick):
    '''
    Is holding the highest card in opening color or biggest trump.
    Returns the card holding the trick so far.
    '''
    # 1 card played means holder is opener
    if len(trick) == 1:
        return trick[0]

    top_trump = 0
    # check if trump holds      
    for card in trick:
        if is_trump(card):
            if card > top_trump:
                top_trump=card
    if top_trump > 0:
        return top_trump

    # find holder in color -> there can be no trump in trick except Trump Fool
    else:
        color=get_color(trick[0]) if get_color(trick[0]) is not 0 else get_color(trick[1])
        # cards are valid if belonging to the color called for
        valid_cards = [card for card in trick if get_color(card) is color]
        sort_cards(valid_cards)
        return valid_cards[-1]


### TESTS ###
for i in range(3):
    trick = random.sample(PLAYING_CARDS, random.randint(2,4))
    print(f"Trick is : {trick}")
    print(f"Holding card is {to_string(holder(trick))}\n")

Trick is : [8, 4]
Holding card is Trump 8

Trick is : ['S9', 'D3', 'HJ', 'D10']
Holding card is Spades 9

Trick is : ['DQ', 'HP', 'SK', 'D5']
Holding card is Diamonds Queen



In [323]:
class Tricks():
    '''
    Class used to easily access data about past tricks.
    '''
    def __init__(self):
        self.tricks=[[]]*18
        self.winner_role=18*[0]        
        self.tricks_color=18*['']
        self.current_trick=0
        
        # ex tricks_color : { 'H' : 0, 'C' : 3, 'D' : 1, 'S' : 0}
        self.color_turns = { color:0 for color in COLOR_VALUES }

        # ex cut_colors : 'TAKER' : { 'H' : False, 'C' : False, 'D' : True, 'S' : False}
        self.cut_colors = { role: { color:False for color in COLOR_VALUES } for role in ROLES }
    
    def add_trick(self,trick,roles):
        turn=self.current_trick
        print(f"[Tricks.add_trick] turn is {turn}")
        self.tricks[turn]=trick
        self.winner_role[turn]=roles[trick.index(holder(trick))]
        print(f"[Tricks.add_trick] holder is {roles[trick.index(holder(trick))]}")        
        self.tricks_color[turn]=get_color(trick[0]) if trick[0]!=0 else get_color(trick[1])
        
        if self.tricks_color[turn] is not None:
            self.color_turns[self.tricks_color[turn]]=self.color_turns[self.tricks_color[turn]]+1

        # update cuts
        if self.tricks_color[turn] is not None:
            for i in range(4):
                if is_trump(trick[i]):
                    self.cut_colors[roles[i]][self.tricks_color[turn]]=True
                    #self.cut_colors[roles[i]][self.tricks_color[turn]]+1
        self.current_trick=turn+1
    
    def to_string(self):
        print(f"\nturn : {self.current_trick}")
        print(f"tricks : {self.tricks}")
        print(f"winner : {self.winner_role}")
        print(f"color : {self.tricks_color}")
        # print(f"color turns : {self.color_turns}")
        print(f"cuts : {self.cut_colors}")
    
    def master(self, color):
        '''
        Returns the strongest card remaining in color
        TODO take the dog into account
        '''
        card_color_in_tricks = [card for trick in self.tricks for card in trick if get_color(card) is color]
        color_cards = [card for card in COLOR_CARDS if get_color(card) is color]

        color_cards.sort(key=get_index,reverse=True)
        for card in color_cards:
            if card not in card_color_in_tricks:
                return card
    
    def trump_master(self):
        '''Returns the trump master card'''
        trumps_in_tricks = [card for trick in self.tricks for card in trick if is_trump(card)]
        trumps=TRUMPS.copy()
        trumps.sort(reverse=True)
        
        for trump in trumps:
            if trump not in trumps_in_tricks:
                return trump
    
    def determine_trump_mastery(self,hand):
        '''
        Returns the number of trumps in hand that are master
        ex: having 21-20-19-17 at game start returns 3 (21-20-19 are untakable, 18 takes 17)
        '''
        trumps_in_tricks = [card for trick in self.tricks for card in trick if is_trump(card)]
        trumps=TRUMPS.copy()
        trumps.sort(reverse=True)
        
        masters=[]
        for trump in trumps:
            if trump not in trumps_in_tricks:
                if trump in hand:
                    masters.append(trump)
                else: 
                    return masters
    
    def compute_final_score(self, ecart):
        # TODO Trump Fool must go back to his owner !!!
        taker_cards=[]
        for card in ecart:
            taker_cards.append(card)
        print(f"reference list is {self.winner_role}")
        for i in range(0,18):
            if self.winner_role[i] == 'TAKER':
                for card in self.tricks[i]:
                    taker_cards.append(card)
        print(f"taker got {self.winner_role.count('TAKER')} tricks\necart={taker_cards[:6]}\ntricks={taker_cards[6:]}")
        return compute_score(taker_cards), get_contract(taker_cards)

winner : ['TAKER', 'TAKER', 'TAKER', 'TAKER', 'TAKER', 'TAKER', 'TAKER', 'TAKER', 'RELAUNCHER', 'RELAUNCHER', 'RELAUNCHER', 'RELAUNCHER', 'RELAUNCHER', 'RELAUNCHER', 'OPENER', 'RELAUNCHER', 'RELAUNCHER', 'TAKER']

### TESTS ###
trick1 = ['S6', 18, 19, 'SP']
roles1 = ['OPENER','TAKER','NEUTRAL','RELAUNCHER']
trick2 = ['C3', 'C4', 'CK', 'C9']
roles2 = ['NEUTRAL','OPENER','TAKER','RELAUNCHER']
trick3 = ['D2', 'D3', 'D1', 'DQ']
roles3 = ['TAKER','RELAUNCHER','NEUTRAL','OPENER']
trick4 = [8, 11, 13, 21]
roles4 = ['OPENER','TAKER','RELAUNCHER','NEUTRAL']

tricks = Tricks()
tricks.add_trick(trick1,roles1)
tricks.add_trick(trick2,roles2)
tricks.add_trick(trick3,roles3)
tricks.add_trick(trick4,roles4)
tricks.to_string()

print(f"\nMaster of Clubs : {tricks.master('C')}")
print(f"Master of Trumps : {tricks.trump_master()}")

hand = [3, 14, 15, 17, 20, 'H4', 'H9', 'HQ', 'D7', 'D10', 'S1']
print(f"\nHand is : {hand}")
print(f"Trump mastery of hand is {tricks.determine_trump_mastery(hand)}\n")

print(tricks.color_turns)

[Tricks.add_trick] turn is 0
[Tricks.add_trick] holder is NEUTRAL
[Tricks.add_trick] turn is 1
[Tricks.add_trick] holder is TAKER
[Tricks.add_trick] turn is 2
[Tricks.add_trick] holder is OPENER
[Tricks.add_trick] turn is 3
[Tricks.add_trick] holder is NEUTRAL

turn : 4
tricks : [['S6', 18, 19, 'SP'], ['C3', 'C4', 'CK', 'C9'], ['D2', 'D3', 'D1', 'DQ'], [8, 11, 13, 21], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
winner : ['NEUTRAL', 'TAKER', 'OPENER', 'NEUTRAL', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
color : ['S', 'C', 'D', None, '', '', '', '', '', '', '', '', '', '', '', '', '', '']
cuts : {'TAKER': {'H': False, 'C': False, 'D': False, 'S': True}, 'OPENER': {'H': False, 'C': False, 'D': False, 'S': False}, 'NEUTRAL': {'H': False, 'C': False, 'D': False, 'S': True}, 'RELAUNCHER': {'H': False, 'C': False, 'D': False, 'S': False}}

Master of Clubs : CQ
Master of Trumps : 20

Hand is : [3, 14, 15, 17, 20, 'H4', 'H9', 'HQ', 'D7', 'D10', 'S1']
Trump mastery of hand is [20, 1

## Biding

In [324]:
CONTRACTS = [56,51,41,36]
POINTS = [1.5,2.5,3.5,4.5]

# CARD + SCORING FUNCTION
def get_card_points(card):
    if is_trump(card):        
        return 4.5 if card in OUDLERS else .5
    else:
        return .5 if card[1] in NON_FIGURES else POINTS[FIGURES.index(card[1])]

# HAND + SCORING FUNCTION
def compute_score(cards):
    return sum(get_card_points(card) for card in cards)

# HAND + SCORING FUNCTION
def get_contract(hand):
    return CONTRACTS[sum([1 for card in hand if is_oudler(card)])]

# HAND + SCORING FUNCTION
def estimate_win_factor(hand):
    '''
    2 essential params : number of trump cards + actual score
    each has .5 potential influence -> the closest to 1, the higher the chances of victory
    '''
    # compare actual points vs needed
    # TODO : check if hands valued under 1 are worth taking
    # score_param = .5 * (1 + (compute_score(hand) + 10 - get_contract(hand)) / get_contract(hand))
    score_param = .5 + .02*(compute_score(hand) + 10 - get_contract(hand))
    
    # ratio of actual trump count VS mean    
    trump_mean = 0 if get_trump_count(hand)==0 else sum([10 if card is 0 else card for card in hand if is_trump(card)]) / get_trump_count(hand)
    # 2 factors : nb_trumps & mean_trumps
    # nb_trumps_factor = 1 + (get_trump_count(hand)-(22/4))/10
    # nb_trumps_factor = 1 + (get_trump_count(hand)/(22/4))/4
    # mean_trumps_factor = 1 + (trump_mean-10) / 20
    # trump_param = .5 * nb_trumps_factor * mean_trumps_factor

    # TODO : biggest trumps have more value
    trump_param = .5 + 0.05*(get_trump_count(hand)-22/4)

    # if score_param + trump_param > 1:
    #     # print(f"score_param : {score_param}, trump_param : {trump_param}\nnb_trumps_factor : {nb_trumps_factor}, mean_trumps_factor : {mean_trumps_factor}")
    #     print(f"score_param : {score_param}, trump_param : {trump_param}")
    return score_param + trump_param


### TESTS ###
# hand1,hand2,hand3,hand4,dog=Game.deal()
# hands = [hand1,hand2,hand3,hand4]

# for i in range(4):
#     print(f"hand {i} has contract {get_contract(hands[i])} with win factor {estimate_win_factor(hands[i])}")
#     print(f"Actual score is {compute_score(hands[i])} and trump count is {get_trump_count(hands[i])}\n")

# for i in range(4):
#     if estimate_win_factor(hands[i]) > 1:
#         print(f"=== Player {i} hand ===")
#         sort_cards(hands[i])
#         show_cards(hands[i])

# good_hand_1 = [1,2,3,6,7,8,9,21, 'HJ','HP','HQ','HK','DJ','DP','DQ','DK','CK','SK']

# print(f"=== Player with good_hand_1 ===")
# print(f"good_hand_1 has contract {get_contract(good_hand_1)} with win factor {estimate_win_factor(good_hand_1)}")
# print(f"Actual score is {compute_score(good_hand_1)} and trump count is {get_trump_count(good_hand_1)}\n")
# sort_cards(good_hand_1)
# show_cards(good_hand_1)

# good_hand_2 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

# print(f"\n=== Player with good hand 2 hand ===")
# print(f"good hand 2 has contract {get_contract(good_hand_2)} with win factor {estimate_win_factor(good_hand_2)}")
# print(f"Actual score is {compute_score(good_hand_2)} and trump count is {get_trump_count(good_hand_2)}\n")
# sort_cards(good_hand_2)
# show_cards(good_hand_2)

## Playing

There are 3 main roles in tarot : Taker - Opener - Relauncher

Taker gameplay :
- play 'longes'
- play figures 'safely'
- make trumps fall

Opener gameplay :
- open and re-open colors where figures are left IF taker doesn't cut

Relauncher gameplay :
- relaunch already opened colors

Neutral can be introduced -> gameplay is between opener and relauncher

In [325]:
class Player():
    def __init__(self, hand, *args, **kwargs):
        '''
        A player is defined by :
        - his role (as taker or in the defense)
        - his hand (initially 18 cards)
        '''
        self.role = kwargs.get('role',None)
        self.hand = hand.copy()


    # TRICK + PLAYER FUNCTION
    def open(self, tricks):
        '''
        Implements the way of opening for each 3 role
        '''
        # masters = [master(color,tricks) for color in COLOR_VALUES]
        masters = [tricks.master(color) for color in COLOR_VALUES]
        print(f"[Player.open] masters are : {masters}")

        if self.role is 'TAKER':
            # try to play strong figures
            # TODO modify ! this is stupid
            for card in self.hand:
                if card in masters:
                    if card is None:
                        print('[Player.open] ERROR 1')
                    return card

            # try to play 'longe'
            for color in COLOR_VALUES:
                color_cards_in_hand = get_color_cards(self.hand,color)
                color_mean = 14-sum([1 for trick in tricks.tricks for card in trick if get_color(card) is color])-len(color_cards_in_hand)/3
                if len(color_cards_in_hand) > color_mean + 3:
                    if get_card_points(card) < 2.5:
                        card=color_cards_in_hand[0]         
                        if card is None:
                            print('[Player.open] ERROR 2')
                        return card

            # make trumps fall
            #*3/7 makes player play his 3rd trump when having 7, which should be acceptable
            # TODO check if trump value is coherent (7<val<12 is optimal)
            trump_count = get_trump_count(self.hand)
            trump_mean = (22 - sum([1 for trick in tricks.tricks for card in trick if is_trump(card)]) - trump_count)/3
            if trump_count >  trump_mean + 3:                
                card=get_trump_cards(self.hand)[int(trump_count*3/7)]
                if card is None:
                    print('[Player.open] ERROR 3')
                return card

            # TODO define what else can be done
            print('[Player.open] Played random card')
            card = random.choice(self.hand)
            if card is None:
                print('[Player.open] ERROR 6')
            return card

        elif self.role is 'OPENER':

            # color_played={color:0 for color in COLOR_VALUES}
            # for trick in tricks.tricks:
            #     get_color(trick[0])

            # opened_colors=[]
            # # try to open new color
            # for trick in tricks.tricks:
            #     for card in trick:
            #         if get_color(card) not in opened_colors and not is_trump(card):
            #             opened_color.append(get_color(card))
            
            tricks.color_turns
            # try to open new color
            # for color in COLOR_VALUES:
            for color in tricks.color_turns:
                # if color not in opened_colors:
                if tricks.color_turns[color] == 0:
                    color_cards_in_hand = get_color_cards(self.hand,color)
                    good_opener = [color_card for color_card in color_cards_in_hand if not is_figure(color_card)]
                    if len(good_opener) > 0:
                        card=good_opener[-1]
                        if card is None:
                            print('[Player.open] ERROR 4')
                        return card
            # TODO define what else can be done
            print('[Player.open] Played random card')
            card = random.choice(self.hand)
            if card is None:
                print('[Player.open] ERROR 6')
            return card
        else:
            # roles other than TAKER and OPENER
            # TODO change this !!!
            print('[Player.open] Played random card')
            card = random.choice(self.hand)
            if card is None:
                print('[Player.open] ERROR 5')
            return card
            
        #     # re-open colors
        #     # TODO manage taker cutting
        #     # TODO manage usefulness of opening (figures left)
        #     for color in hand:
        #         if figures fall count < 3:
        #             return re-open color

        #     # TODO define what else can be done
        #     return random.choice(hand)

        # elif self.role is 'RELAUNCHER':
        #     # try to relaunch color
        #     # try to get information if figures in color


    # TRICK + PLAYER FUNCTION
    def play_color(self,cards_in_color,tricks):
        '''
        Player has to answer in color if he can except if he has Trump Fool.
        He might want to take risk, or not.        
        '''
        # TODO change random choice 
        return random.choice(cards_in_color)
        
    
    # TRICK + PLAYER FUNCTION
    def cutOrClimb(self,current_trick,taker_id):
        '''Player has to trump or overtrump.'''    
        player_trumps = get_trump_cards(self.hand)

        # player has 1 trump : he must play it
        # TODO take fool into consideration
        if len(player_trumps) is 1:
            return player_trumps[0]

        trick_trumps = [card for card in current_trick if is_trump(card)]
        ref = max(trick_trumps) if len(trick_trumps) > 0 else 0
        
        # TODO define what is safe to TAKER
        # safe = False if taker_id > len(current_trick) else not holder(current_trick)==current_trick[taker_id]
        safe = False if taker_id > len(current_trick) else True if taker_id==len(current_trick) else not holder(current_trick)==current_trick[taker_id]

        # climb or cut
        if max(player_trumps) > ref:
            # try play 1
            if 1 in player_trumps and safe:
                return 1
            for trump in player_trumps:
                if trump > ref: return trump
        # discard
        else:
            #try play 1
            if 1 in player_trumps and safe:
                return 1
            #discard anything
            else:
                sort_cards(player_trumps)
                for card in player_trumps:
                    if card != 1:
                        return card
                return player_trumps[0]
    
    def play_card(self,current_trick,taker_id,tricks,**kwargs):
        '''
        Returns the choice of a card from the player.
        Removes the chosen card from his hand.
        '''
        is_first_player=kwargs.get("is_first",False)
        if is_first_player:
            card = self.open(tricks)
            if card is None:
                print(f"[Player.play_card] ERROR IN OPEN FUNC -> returned NONE")
        else:
            card = self.answer(current_trick,taker_id,tricks)
            if card is None:                
                print(f"[Player.play_card] ERROR IN ANSWER FUNC -> returned NONE")
        print(f"[Player.play_card] player hand is : {self.hand}")
        print(f"[Player.play_card] chosen card is : {card}")
        self.hand.remove(card)
        return card

    # TRICK + PLAYER FUNCTION
    def discard(self,current_trick,taker_id,tricks):
        '''
        Never safe to discard as TAKER.
        Safe to discard if TAKER not holding.
        If SAFE -> Policy is to discard FIGURES with priority given to cut colors.
        '''
        safe = False if taker_id >= len(current_trick) else not holder(current_trick)==current_trick[taker_id]

        #! TypeError: '>=' not supported between instances of 'Tricks' and 'int'
        
        if safe:
            # TODO is discarding figures the best option ? what if taker out of trumps ?
            # discard figure in color cut by TAKER
            for color in tricks.cut_colors['TAKER']:
                if tricks.cut_colors['TAKER'][color]:
                    color_cards=get_color_cards(self.hand,color)
                    # check if player has cards in cut colors
                    if len(color_cards)>0:
                        color_cards.sort(key=get_index,reverse=True)
                        # play figures
                        for card in color_cards:
                            if is_figure(card):
                                print('[discard] player chose to discard in cut color')
                                return card
            #TODO get an idea of the game state : losing ? winning ? if losing then discard other figures !
            # for now we simply discard any possible figure
            for color in COLOR_VALUES:
                color_cards=get_color_cards(self.hand,color)
                # check if player has cards in cut colors
                if len(color_cards)>0:
                    color_cards.sort(key=get_index,reverse=True)
                    # play figures
                    for card in color_cards:
                        if is_figure(card):
                            print('[discard] player chose to discard figure')
                            return card
            # TODO default will be discarding lowest -> get more value
        else:
            # TODO think about what to do if unsafe ?
            # discard card in color cut by TAKER
            pass
        
        # player has nothing of value : he discards his lowest card
        lowest=self.hand[0]
        for card in self.hand:
            if get_card_value(card)<get_card_value(lowest):
                lowest = card
        print('[discard] player chose to discard his lowest card')
        return lowest

        # TRICK + PLAYER FUNCTION
    def answer(self, current_trick, taker_id,tricks):
        '''
        If a player doesn't open, he has to answer to the opening card
        '''
        opening = current_trick[0]

        if is_trump(opening):
            # trump opening is managed
            if get_trump_count(self.hand) >0:
                return self.cutOrClimb(current_trick,taker_id) if opening!=0 else open(tricks)
            else:
                return self.discard(current_trick,taker_id,tricks)
        else:
            # answer in color
            color=get_color(opening)
            cards_in_color=get_color_cards(color,self.hand)
            if len(cards_in_color)>0:
                # TODO compute risk of playing figure
                return self.play_color(cards_in_color,tricks)

            # cut : player has trumps, he must use them
            elif get_trump_count(self.hand) > 0:
                return self.cutOrClimb(current_trick,taker_id)

            # discard : player has no trump, nor cards in color
            else:
                return self.discard(current_trick,taker_id,tricks)

In [326]:
### TESTS over PLAYER ###
for i in range(5):
    sampling = random.sample(PLAYING_CARDS, random.randint(12,14))
    hand=sampling[:10]
    sort_cards(hand)
    myPlayer=Player(hand)
    trick=sampling[10:]
    if get_trump_count(hand)>0:
        print(f"\nHand : {hand}")
        print(f"Trick : {trick}")
        print(f"Choice: {myPlayer.cutOrClimb(trick,1)}")

trick1 = ['S6', 18, 19, 'SP']
roles1 = ['OPENER','TAKER','NEUTRAL','RELAUNCHER']
trick2 = ['C3', 'C4', 'CK', 'C9']
roles2 = ['NEUTRAL','OPENER','TAKER','RELAUNCHER']
trick3 = ['D2', 'D3', 'D1', 'DQ']
roles3 = ['TAKER','RELAUNCHER','NEUTRAL','OPENER']
trick4 = [8, 11, 13, 21]
roles4 = ['OPENER','TAKER','RELAUNCHER','NEUTRAL']

tricks = Tricks()
tricks.add_trick(trick1,roles1)
tricks.add_trick(trick2,roles2)
tricks.add_trick(trick3,roles3)
tricks.add_trick(trick4,roles4)
tricks.to_string()

print()
hand = ['H4', 'H9', 'HQ', 'D7', 'D10', 'S1']
myPlayer = Player(hand,role='NEUTRAL')
print(hand)
print(myPlayer.discard(['S8', 2,'SP'],1,tricks))

hand = ['H4', 'H9', 'D7', 'D10', 'S1']
myPlayer = Player(hand,role='NEUTRAL')
print(hand)
print(myPlayer.discard(['S8', 2,'SP'],1,tricks))

hand = ['H4', 'H9', 'D7', 'D10', 'S1']
myPlayer = Player(hand,role='NEUTRAL')
print(hand)
print(myPlayer.answer(['S8', 2,'SP'],1,tricks))


Hand : [11, 13, 'H4', 'H5', 'H8', 'C5', 'C6', 'DQ', 'S2', 'S5']
Trick : ['C7', 4]
Choice: 11

Hand : [9, 10, 14, 17, 19, 'H7', 'HK', 'C1', 'CK', 'D5']
Trick : [13, 'S10', 20, 'C8']
Choice: 9

Hand : [0, 2, 21, 'H8', 'H10', 'D1', 'D10', 'S5', 'S9', 'S10']
Trick : ['C9', 'S6', 17, 'S3']
Choice: 21

Hand : [5, 7, 8, 20, 'HQ', 'HK', 'CQ', 'D10', 'S1', 'S4']
Trick : ['C10', 'D5', 'D1']
Choice: 5

Hand : [0, 1, 12, 'H1', 'H5', 'HQ', 'C9', 'DQ', 'S9', 'SJ']
Trick : [10, 'CQ']
Choice: 1
[Tricks.add_trick] turn is 0
[Tricks.add_trick] holder is NEUTRAL
[Tricks.add_trick] turn is 1
[Tricks.add_trick] holder is TAKER
[Tricks.add_trick] turn is 2
[Tricks.add_trick] holder is OPENER
[Tricks.add_trick] turn is 3
[Tricks.add_trick] holder is NEUTRAL

turn : 4
tricks : [['S6', 18, 19, 'SP'], ['C3', 'C4', 'CK', 'C9'], ['D2', 'D3', 'D1', 'DQ'], [8, 11, 13, 21], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
winner : ['NEUTRAL', 'TAKER', 'OPENER', 'NEUTRAL', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [340]:
class Game():
    """DOC"""

    def __init__(self, *args, **kwargs):
        """DOC"""
        hand0,hand1,hand2,hand3,dog=self.deal()
        self.hands = [hand0,hand1,hand2,hand3]
        for hand in self.hands:
            sort_cards(hand)
        self.dog = dog
        self.ecart=6*[0]
        self.players = [Player(hand) for hand in self.hands]
        
        self.dealer = 0
        self.first_player = 1
        self.tricks=Tricks()

    # GAME FUNCTION
    def deal(self):
        '''Returns 4 hands of 18 cards + a dog of 6 cards'''
        deck = PLAYING_CARDS.copy()
        random.shuffle(deck)
        return [deck[0:18],deck[18:36],deck[36:54],deck[54:72],deck[72:78]]

    # GAME FUNCTION
    def bid(self):
        '''
        Computes win_factor for each player
        Returns taker_id, announces
        '''
        current_bid=BID_VALUES[0]
        player_bids = [0,0,0,0]
        for i in range(4):
            win_factor = estimate_win_factor(self.players[i].hand)
            if win_factor<1:
                pass
            else:
                # TAKE
                if win_factor < QUANTILES[3] and current_bid<BID_VALUES[1]:
                    player_bids[i]=BID_VALUES[1]
                    current_bid=BID_VALUES[1]
                # PUSH
                elif win_factor < QUANTILES[5] and current_bid<BID_VALUES[2]:
                    player_bids[i]=BID_VALUES[2]
                    current_bid=BID_VALUES[2]
                # GUARD
                elif win_factor < QUANTILES[7] and current_bid<BID_VALUES[3]:
                    player_bids[i]=BID_VALUES[3]
                    current_bid=BID_VALUES[3]
                # GUARD WITHOUT
                elif win_factor < QUANTILES[8] and current_bid<BID_VALUES[4]:
                    player_bids[i]=BID_VALUES[4]
                    current_bid=BID_VALUES[4]
                # GUARD AGAINST
                elif current_bid<BID_VALUES[5]:
                    player_bids[i]=BID_VALUES[5]
                    current_bid=BID_VALUES[5]
                    
        if max(player_bids) is 0: 
            print("No one took.")
        else:
            for i in range(4):
                self.players[(player_bids.index(max(player_bids))-i)%4].role=ROLES[i%4]
        
        return player_bids.index(max(player_bids)),[BIDS[BID_VALUES.index(bid)] for bid in player_bids]

    def incorporate_dog(self,player):
        '''
        Add 6 dog cards to TAKER hand.
        Then removes 6 cards from TAKER hand.
        '''
        for card in self.dog:
            player.hand.append(card)

        # TODO do a better ecart
        ecart_count=0        
        while ecart_count<6:
            card=random.choice(player.hand)
            # pick random cards except kings and trumps
            if not is_trump(card) and not get_card_role(card)=='K':
                self.ecart[ecart_count]=card
                player.hand.remove(card)
                ecart_count=ecart_count+1


    # GAME FUNCTION
    def play_trick(self):
        '''
        Each player plays one card.
        First player is opener, the rest simply answer.
        The strongest determines the trick's winner.
        '''
        current_trick=[]
        roles=[0,0,0,0]
        if len(self.players[0].hand) == 18:
            self.first_player = (self.dealer+1)%4

        print("\n[Game.play_trick] === NEW TRICK ===")
        for i in range(4):
            # players should play in the right order
            player_id=(self.first_player+i)%4
            player=self.players[player_id]
            taker_turn=(self.taker_id+4-self.first_player)%4+1
            # print("[Game.play_trick] displaying roles")
            print(f"[Game.play_trick] player_id is {player_id}")
            print(f"[Game.play_trick] current player is {player.role}")
            roles[i]=player.role
            print(f"[Game.play_trick] taker will play as {taker_turn}")
            chosen_card=self.players[(self.first_player+i)%4].play_card(current_trick,taker_turn-1,self.tricks,is_first=((self.first_player+i)%4==self.first_player))
            print(f"[Game.play_trick] chosen card : {chosen_card}")
            current_trick.append(chosen_card)
        print(f"[Game.play_trick] called function [tricks.add_trick] over trick={current_trick} and roles {roles}")
        self.tricks.add_trick(current_trick,roles)
        print(f"winner_role list is now : {self.tricks.winner_role}")

### TESTS ###
myGame = Game()
bids = myGame.bid()
print(f"[Game.TEST] bids are {bids}")

if bids[1].count('PASS')!=4:
    for player in myGame.players:
        if player.role is 'TAKER':
            myGame.taker_id=myGame.players.index(player)
            # print(f"[Game.__init__]found {player.role}, id is {self.taker_id}")
        print(f"[Game.TEST] displaying roles {player.role}")
        
    myGame.incorporate_dog(myGame.players[myGame.taker_id])
    print(f"TAKER has elected ecart={myGame.ecart}")
    myTakerHand=myGame.players[myGame.taker_id].hand
    sort_cards(myTakerHand)
    print(f"TAKER hand is {myTakerHand}")

    for i in range(18):
        myGame.play_trick()
    myGame.tricks.to_string()
    score,contract=myGame.tricks.compute_final_score(myGame.ecart)
    print(f'\nTaker final score is {score} with contract {contract}')
# print(player.role for player in myGame.players)

[Game.TEST] bids are (3, ['PASS', 'PASS', 'PASS', 'GUARD AGAINST'])
[Game.TEST] displaying roles RELAUNCHER
[Game.TEST] displaying roles NEUTRAL
[Game.TEST] displaying roles OPENER
[Game.TEST] displaying roles TAKER
TAKER has elected ecart=['D4', 'D10', 'C5', 'CQ', 'SP', 'S1']
TAKER hand is [0, 1, 2, 5, 12, 14, 21, 'H3', 'H4', 'HK', 'C2', 'C7', 'CK', 'D7', 'D9', 'DP', 'S2', 'SK']

[Game.play_trick] === NEW TRICK ===
[Game.play_trick] player_id is 1
[Game.play_trick] current player is NEUTRAL
[Game.play_trick] taker will play as 3
[Player.open] masters are : ['HK', 'CK', 'DK', 'SK']
[Player.open] Played random card
[Player.play_card] player hand is : [4, 7, 8, 10, 15, 17, 'H7', 'HJ', 'C3', 'C6', 'C8', 'D2', 'D6', 'D8', 'S4', 'S5', 'S6', 'S7']
[Player.play_card] chosen card is : 4
[Game.play_trick] chosen card : 4
[Game.play_trick] player_id is 2
[Game.play_trick] current player is OPENER
[Game.play_trick] taker will play as 3
[Player.play_card] player hand is : [9, 13, 18, 19, 20, 'H9',